In [46]:
# Imports
import pandas as pd
import os
import typing
from functools import partial
from pprint import pprint
from pypif import pif

In [47]:
# Load Data
# Filter to comp
# Parse Vector
# Upload to citrination

In [48]:
load_path = os.path.join('data','training_data.csv')
save_path = os.path.join('data','processed_data.csv')

In [49]:
df = pd.read_csv(load_path)

In [50]:
original_cols = df.columns

In [51]:
cols = ['{}_percent'.format(i*10) for i in range(11)]

In [52]:
def debug(func):
    def debug_wrapper(*args, **kwargs):
        row = args[0]
        print(row['stabilityVec'])
        func(*args, **kwargs)
    return debug_wrapper

In [53]:
def vec_to_stability(row: pd.Series, cols: list) -> pd.Series:

    vec = eval(row['stabilityVec'])
    for element, col in zip(vec, cols):
        row[col] = int(element)
    
    return row

In [54]:
vtf = partial(vec_to_stability, cols=cols)

In [55]:
df = df.apply(vtf, axis=1)

## Pivot each row and convert the stability vector into a chemical formula 
Pivot stability vector

In [56]:
binary_df = df.melt(id_vars=original_cols, var_name='weight_fraction_element_b', value_name='stable')

In [57]:
binary_df['weight_fraction_element_b'] = binary_df['weight_fraction_element_b'].str.strip('_percent').astype(float)

In [58]:
binary_df['weight_fraction_element_b'] = binary_df['weight_fraction_element_b']/100

Convert `weight_percent` to formula

In [60]:
def row_to_formula(row: pd.Series):
    w = row['weight_fraction_element_b']
    a = row['formulaA']
    b = row['formulaB']
    
    if w == 0:
        return a
    elif w == 100:
        return b
    else:
        wa = 1.0-w
        def compute_formula(wa):
            return f"{a}{wa:.1f} {b}{w:.1f}"
        return compute_formula(wa)
             

In [61]:
binary_df['formula'] = binary_df.apply(row_to_formula, axis=1)

Export data

In [62]:
# only export unique
binary_df = binary_df.drop_duplicates('formula')

In [64]:
binary_df[['formula','formulaA','formulaB','stable']].to_csv(save_path, index=False)